In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import date

#오류 무시
import warnings
warnings.filterwarnings('ignore')

In [26]:
#데이터 불러오기
covid = pd.read_csv('./data/owid-covid-data.csv')

movie2018 = pd.read_csv('./data/movie2018.csv', encoding = 'utf-8')
movie2019 = pd.read_csv('./data/movie2019.csv', encoding = 'utf-8')
movie2020 = pd.read_csv('./data/movie2020.csv', encoding = 'utf-8')
movie2021 = pd.read_csv('./data/movie2021.csv', encoding = 'utf-8')
movie2022 = pd.read_csv('./data/movie2022.csv', encoding = 'utf-8')

dust2018 = pd.read_csv('./data/dust_2018.csv', encoding = 'euc-kr')
dust2019 = pd.read_csv('./data/dust_2019.csv', encoding = 'euc-kr')
dust2020 = pd.read_csv('./data/dust_2020.csv', encoding = 'euc-kr')
dust2021 = pd.read_csv('./data/dust_2021.csv', encoding = 'euc-kr')

weathers = pd.read_csv('./data/weathers.csv', encoding = 'euc-kr')
holiday = pd.read_csv('./data/holiday.csv', encoding = 'euc-kr')

### 1. 코로나 데이터 정제

In [3]:
#covid 데이터에서 south korea 및 필요한 변수만 추출
covid_kor = covid[covid['location'] == 'South Korea'][['date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths',
                                                       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 
                                                       'new_vaccinations', 'people_vaccinated_per_hundred',
                                                       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred']]

#20200120-20200122 데이터 추가
covid_kor.loc[0] = ['2020-01-20', 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
covid_kor.loc[1] = ['2020-01-21', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

covid_kor.reset_index(drop = True, inplace = True)
covid_kor

,date,total_cases,new_cases,total_deaths,new_deaths,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,2020-01-22,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-25,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-26,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
741,2022-02-01,884310.0,20268.0,6787.0,15.0,44645621.0,44000950.0,27259463.0,2987.0,87.02,85.76,53.13
742,2022-02-02,907214.0,22904.0,6812.0,25.0,44646116.0,44001773.0,27271785.0,13640.0,87.02,85.76,53.16
743,2022-02-03,934656.0,27442.0,6836.0,24.0,44658333.0,44053608.0,27594934.0,387165.0,87.04,85.87,53.79
744,2020-01-20,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00


In [76]:
#날짜 타입으로 변환
covid_kor['date'] = covid_kor['date'].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d'))
covid_kor['date'] = covid_kor['date'].dt.date

In [4]:
#결측치 확인 및 0으로 처리
print(covid_kor.isnull().sum())
covid_kor.fillna(0, inplace = True)

date                                     0
total_cases                              0
new_cases                                1
total_deaths                            29
new_deaths                              29
people_vaccinated                      401
people_fully_vaccinated                401
total_boosters                         444
new_vaccinations                       402
people_vaccinated_per_hundred          401
people_fully_vaccinated_per_hundred    401
total_boosters_per_hundred             444
dtype: int64


### 2. 영화 데이터 정제

In [5]:
#전체 데이터 합치기
movies = pd.concat([movie2018, movie2019, movie2020, movie2021, movie2022], axis = 0)
movies.reset_index(drop = True, inplace = True)
movies

,날짜,개봉편수_kor,상영편수_kor,매출액_kor,관객수_kor,점유율_kor,개봉편수_for,상영편수_for,매출액_for,관객수_for,점유율_for,개봉편수_all,상영편수_all,매출액_all,관객수_all
0,2018-01-01,0,20,"13,235,297,400","1,577,178",90.60%,0,58,"1,342,794,500","164,603",9.50%,0,78,"14,578,091,900","1,741,781"
1,2018-01-02,1,25,"4,263,314,700","567,998",85.90%,0,55,"694,920,400","93,207",14.10%,1,80,"4,958,235,100","661,205"
2,2018-01-03,0,20,"3,217,243,500","426,349",64.60%,4,55,"1,718,812,900","234,025",35.40%,4,75,"4,936,056,400","660,374"
3,2018-01-04,1,17,"3,102,854,500","411,457",65.00%,14,64,"1,625,490,400","221,393",35.00%,15,81,"4,728,344,900","632,850"
4,2018-01-05,0,17,"4,216,406,540","501,481",67.70%,0,51,"1,965,267,057","239,263",32.30%,0,68,"6,181,673,597","740,744"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487,2022-01-27,7,34,"887,839,220","97,737",79.40%,5,49,"212,678,380","25,301",20.60%,12,83,"1,100,517,600","123,038"
1488,2022-01-28,1,27,"1,074,282,700","109,860",75.80%,2,53,"308,807,600","35,079",24.20%,3,80,"1,383,090,300","144,939"
1489,2022-01-29,0,24,"2,105,911,290","212,515",79.90%,0,48,"489,717,700","53,360",20.10%,0,72,"2,595,628,990","265,875"
1490,2022-01-30,1,25,"2,245,669,730","225,105",79.50%,0,51,"532,860,030","58,045",20.50%,1,76,"2,778,529,760","283,150"


In [6]:
#2018-01-20 ~ 2022-01-20으로 기간 조정
movies = movies.iloc[19:1481]
movies

,날짜,개봉편수_kor,상영편수_kor,매출액_kor,관객수_kor,점유율_kor,개봉편수_for,상영편수_for,매출액_for,관객수_for,점유율_for,개봉편수_all,상영편수_all,매출액_all,관객수_all
19,2018-01-20,0,23,"5,185,645,648","615,954",48.10%,0,52,"5,547,126,635","664,917",51.90%,0,75,"10,732,772,283","1,280,871"
20,2018-01-21,0,26,"4,859,662,600","578,595",48.80%,0,55,"5,055,412,800","608,086",51.20%,0,81,"9,915,075,400","1,186,681"
21,2018-01-22,1,27,"1,571,629,100","207,824",51.50%,0,50,"1,483,313,100","195,857",48.50%,1,77,"3,054,942,200","403,681"
22,2018-01-23,1,27,"1,585,247,700","210,219",51.70%,1,58,"1,472,570,900","196,435",48.30%,2,85,"3,057,818,600","406,654"
23,2018-01-24,5,27,"1,537,700,100","203,559",52.20%,4,63,"1,397,108,500","186,699",47.80%,9,90,"2,934,808,600","390,258"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476,2022-01-16,0,18,"1,054,717,200","103,939",36.80%,0,49,"1,780,404,210","178,716",63.20%,0,67,"2,835,121,410","282,655"
1477,2022-01-17,0,21,"302,660,960","31,364",36.10%,4,48,"501,215,360","55,458",63.90%,4,69,"803,876,320","86,822"
1478,2022-01-18,1,27,"354,170,910","36,952",36.20%,2,49,"581,098,400","65,120",63.80%,3,76,"935,269,310","102,072"
1479,2022-01-19,1,23,"352,398,380","37,032",35.20%,4,51,"612,495,770","68,235",64.80%,5,74,"964,894,150","105,267"


In [73]:
#날짜 타입으로 변환
movies['날짜'] = movies['날짜'].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d'))
movies['날짜'] = movies['날짜'].dt.date

In [7]:
#영화데이터 결측치 X
movies.isnull().sum()

날짜          0
개봉편수_kor    0
상영편수_kor    0
매출액_kor     0
관객수_kor     0
점유율_kor     0
개봉편수_for    0
상영편수_for    0
매출액_for     0
관객수_for     0
점유율_for     0
개봉편수_all    0
상영편수_all    0
매출액_all     0
관객수_all     0
dtype: int64

### 3. 날씨 데이터 정제

In [59]:
#강수량, 적설량 결측치 0으로 대체
weathers.fillna(0, inplace = True)
weathers

,지점,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),일 최심신적설(cm),일 최심적설(cm)
0,90,속초,2018-01-20,4.2,1.2,7.0,0.0,5.0,2.2,41.4,0.0,0.0
1,90,속초,2018-01-21,2.1,-0.8,5.5,0.0,5.3,1.8,31.8,0.0,0.0
2,90,속초,2018-01-22,0.0,-2.8,3.1,2.4,4.7,2.0,66.6,0.0,0.0
3,90,속초,2018-01-23,-9.0,-13.8,-1.7,0.0,8.0,2.9,30.3,0.0,0.0
4,90,속초,2018-01-24,-12.3,-15.9,-8.4,0.0,6.1,2.4,23.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
138947,295,남해,2022-01-16,3.4,-1.3,8.0,0.0,4.9,2.1,56.8,0.0,0.0
138948,295,남해,2022-01-17,1.6,-3.1,5.2,0.0,5.7,2.6,51.6,0.0,0.0
138949,295,남해,2022-01-18,-1.1,-5.6,4.4,0.0,5.5,1.4,40.6,0.0,0.0
138950,295,남해,2022-01-19,1.2,-5.0,6.8,0.0,4.9,1.6,41.4,0.0,0.0


In [60]:
#일자별로 각 칼럼의 평균값 구하기
new_weathers = weathers.groupby(['일시']).mean()
new_weathers.drop(columns = ['지점'], inplace = True)
new_weathers.reset_index(drop = False, inplace = True)
new_weathers

,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),일 최심신적설(cm),일 최심적설(cm)
0,2018-01-20,1.284211,-3.844211,7.565263,0.000000,4.089474,1.457895,64.489474,0.000000,0.000000
1,2018-01-21,1.311579,-3.871579,7.481053,0.000000,3.431579,1.191579,60.866316,0.000000,0.000000
2,2018-01-22,2.971579,-0.541053,7.477895,1.304211,6.088421,2.026316,65.282105,0.146316,0.146316
3,2018-01-23,-6.051579,-9.615789,0.953684,0.064211,7.265263,4.091579,48.290526,0.046316,0.170526
4,2018-01-24,-9.991579,-13.110526,-6.069474,0.206316,6.877895,3.460000,41.976842,0.303158,0.400000
...,...,...,...,...,...,...,...,...,...,...
1457,2022-01-16,0.997895,-3.826316,5.397895,0.078947,6.332632,2.932632,54.527368,0.000000,0.236842
1458,2022-01-17,-1.933684,-6.302105,2.733684,0.132632,6.289474,2.768421,54.464211,0.069474,0.238947
1459,2022-01-18,-3.614737,-8.514737,2.106316,0.027368,4.697895,2.003158,46.534737,0.049474,0.236842
1460,2022-01-19,-2.310526,-8.470526,3.992632,0.541053,5.502105,1.915789,59.974737,0.261053,0.462105


In [61]:
#날짜 타입으로 변환
new_weathers['일시'] = new_weathers['일시'].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d'))
new_weathers['일시'] = new_weathers['일시'].dt.date

### 4. 미세먼지 데이터 정제

In [10]:
#미세먼지 데이터 합치기
dust = pd.concat([dust2018, dust2019, dust2020, dust2021], axis = 0)
dust.reset_index(drop = True, inplace = True)
dust

,지점,지점명,일시,1시간평균 미세먼지농도(㎍/㎥)
0,90,속초,2018-01-20 01:00,59
1,90,속초,2018-01-20 02:00,59
2,90,속초,2018-01-20 03:00,64
3,90,속초,2018-01-20 04:00,82
4,90,속초,2018-01-20 05:00,87
...,...,...,...,...
925014,501,연평도,2022-01-20 19:00,14
925015,501,연평도,2022-01-20 20:00,15
925016,501,연평도,2022-01-20 21:00,15
925017,501,연평도,2022-01-20 22:00,29


In [11]:
#일시를 시간 타입으로 변환
dust['일시'] = dust['일시'].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d %H:%M'))
dust['newdate'] = dust['일시'].dt.date

In [12]:
#일자별 데이터로 변경
newdust = dust[['newdate', '1시간평균 미세먼지농도(㎍/㎥)']].groupby(['newdate']).mean()
newdust.reset_index(drop = False, inplace = True)
newdust

,newdate,1시간평균 미세먼지농도(㎍/㎥)
0,2018-01-20,73.992089
1,2018-01-21,60.352941
2,2018-01-22,42.562404
3,2018-01-23,21.779160
4,2018-01-24,21.806015
...,...,...
1456,2022-01-16,32.306569
1457,2022-01-17,21.027314
1458,2022-01-18,24.195266
1459,2022-01-19,30.997028


### 5. 공휴일 데이터 정제

In [40]:
#locdate를 날짜형식으로 변경
newhol = pd.DataFrame()

newhol['locdate'] = holiday['locdate'].astype(str)
newhol['locdate'] = newhol['locdate'].apply(lambda x : datetime.datetime.strptime(x, '%Y%m%d'))
newhol['locdate'] = newhol['locdate'].dt.date
newhol

,locdate
0,2018-01-01
1,2018-02-15
2,2018-02-16
3,2018-02-17
4,2018-03-01
...,...
67,2021-10-09
68,2021-10-11
69,2021-12-25
70,2022-01-01


In [41]:
#공휴일 여부를 나타내는 y칼럼 생성
newhol['y'] = 1

### 전체 데이터 병합

In [65]:
data1 = pd.merge(new_weathers, newdust, how = 'outer', left_on = '일시', right_on = 'newdate')
data1.drop(columns = ['newdate'], inplace = True)
data1

,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),일 최심신적설(cm),일 최심적설(cm),1시간평균 미세먼지농도(㎍/㎥)
0,2018-01-20,1.284211,-3.844211,7.565263,0.000000,4.089474,1.457895,64.489474,0.000000,0.000000,73.992089
1,2018-01-21,1.311579,-3.871579,7.481053,0.000000,3.431579,1.191579,60.866316,0.000000,0.000000,60.352941
2,2018-01-22,2.971579,-0.541053,7.477895,1.304211,6.088421,2.026316,65.282105,0.146316,0.146316,42.562404
3,2018-01-23,-6.051579,-9.615789,0.953684,0.064211,7.265263,4.091579,48.290526,0.046316,0.170526,21.779160
4,2018-01-24,-9.991579,-13.110526,-6.069474,0.206316,6.877895,3.460000,41.976842,0.303158,0.400000,21.806015
...,...,...,...,...,...,...,...,...,...,...,...
1457,2022-01-16,0.997895,-3.826316,5.397895,0.078947,6.332632,2.932632,54.527368,0.000000,0.236842,32.306569
1458,2022-01-17,-1.933684,-6.302105,2.733684,0.132632,6.289474,2.768421,54.464211,0.069474,0.238947,21.027314
1459,2022-01-18,-3.614737,-8.514737,2.106316,0.027368,4.697895,2.003158,46.534737,0.049474,0.236842,24.195266
1460,2022-01-19,-2.310526,-8.470526,3.992632,0.541053,5.502105,1.915789,59.974737,0.261053,0.462105,30.997028


In [67]:
newhol

,locdate,y
0,2018-01-01,1
1,2018-02-15,1
2,2018-02-16,1
3,2018-02-17,1
4,2018-03-01,1
...,...,...
67,2021-10-09,1
68,2021-10-11,1
69,2021-12-25,1
70,2022-01-01,1


In [70]:
data2 = pd.merge(data1, newhol, how = 'outer', left_on = '일시', right_on = 'locdate')
data2.drop(columns = 'locdate', inplace = True)
data2.fillna(0, inplace = True)

#기간 내에 해당되지 않는 칼럼 제거
data2.drop(index = [1462, 1463], inplace = True)
data2

,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),일 최심신적설(cm),일 최심적설(cm),1시간평균 미세먼지농도(㎍/㎥),y
0,2018-01-20,1.284211,-3.844211,7.565263,0.000000,4.089474,1.457895,64.489474,0.000000,0.000000,73.992089,0.0
1,2018-01-21,1.311579,-3.871579,7.481053,0.000000,3.431579,1.191579,60.866316,0.000000,0.000000,60.352941,0.0
2,2018-01-22,2.971579,-0.541053,7.477895,1.304211,6.088421,2.026316,65.282105,0.146316,0.146316,42.562404,0.0
3,2018-01-23,-6.051579,-9.615789,0.953684,0.064211,7.265263,4.091579,48.290526,0.046316,0.170526,21.779160,0.0
4,2018-01-24,-9.991579,-13.110526,-6.069474,0.206316,6.877895,3.460000,41.976842,0.303158,0.400000,21.806015,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1457,2022-01-16,0.997895,-3.826316,5.397895,0.078947,6.332632,2.932632,54.527368,0.000000,0.236842,32.306569,0.0
1458,2022-01-17,-1.933684,-6.302105,2.733684,0.132632,6.289474,2.768421,54.464211,0.069474,0.238947,21.027314,0.0
1459,2022-01-18,-3.614737,-8.514737,2.106316,0.027368,4.697895,2.003158,46.534737,0.049474,0.236842,24.195266,0.0
1460,2022-01-19,-2.310526,-8.470526,3.992632,0.541053,5.502105,1.915789,59.974737,0.261053,0.462105,30.997028,0.0


In [75]:
data3 = pd.merge(data2, movies, how = 'outer', left_on = '일시', right_on = '날짜')
data3.drop(columns = '날짜', inplace = True)
data3

,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),일 최심신적설(cm),일 최심적설(cm),...,점유율_kor,개봉편수_for,상영편수_for,매출액_for,관객수_for,점유율_for,개봉편수_all,상영편수_all,매출액_all,관객수_all
0,2018-01-20,1.284211,-3.844211,7.565263,0.000000,4.089474,1.457895,64.489474,0.000000,0.000000,...,48.10%,0,52,"5,547,126,635","664,917",51.90%,0,75,"10,732,772,283","1,280,871"
1,2018-01-21,1.311579,-3.871579,7.481053,0.000000,3.431579,1.191579,60.866316,0.000000,0.000000,...,48.80%,0,55,"5,055,412,800","608,086",51.20%,0,81,"9,915,075,400","1,186,681"
2,2018-01-22,2.971579,-0.541053,7.477895,1.304211,6.088421,2.026316,65.282105,0.146316,0.146316,...,51.50%,0,50,"1,483,313,100","195,857",48.50%,1,77,"3,054,942,200","403,681"
3,2018-01-23,-6.051579,-9.615789,0.953684,0.064211,7.265263,4.091579,48.290526,0.046316,0.170526,...,51.70%,1,58,"1,472,570,900","196,435",48.30%,2,85,"3,057,818,600","406,654"
4,2018-01-24,-9.991579,-13.110526,-6.069474,0.206316,6.877895,3.460000,41.976842,0.303158,0.400000,...,52.20%,4,63,"1,397,108,500","186,699",47.80%,9,90,"2,934,808,600","390,258"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,2022-01-16,0.997895,-3.826316,5.397895,0.078947,6.332632,2.932632,54.527368,0.000000,0.236842,...,36.80%,0,49,"1,780,404,210","178,716",63.20%,0,67,"2,835,121,410","282,655"
1458,2022-01-17,-1.933684,-6.302105,2.733684,0.132632,6.289474,2.768421,54.464211,0.069474,0.238947,...,36.10%,4,48,"501,215,360","55,458",63.90%,4,69,"803,876,320","86,822"
1459,2022-01-18,-3.614737,-8.514737,2.106316,0.027368,4.697895,2.003158,46.534737,0.049474,0.236842,...,36.20%,2,49,"581,098,400","65,120",63.80%,3,76,"935,269,310","102,072"
1460,2022-01-19,-2.310526,-8.470526,3.992632,0.541053,5.502105,1.915789,59.974737,0.261053,0.462105,...,35.20%,4,51,"612,495,770","68,235",64.80%,5,74,"964,894,150","105,267"


In [84]:
final = pd.merge(data3, covid_kor, how = 'outer', left_on = '일시', right_on = 'date')
final.drop(columns = 'date', inplace = True)
final.fillna(0, inplace = True)

final

,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),일 최심신적설(cm),일 최심적설(cm),...,new_cases,total_deaths,new_deaths,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,2018-01-20,1.284211,-3.844211,7.565263,0.000000,4.089474,1.457895,64.489474,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
1,2018-01-21,1.311579,-3.871579,7.481053,0.000000,3.431579,1.191579,60.866316,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
2,2018-01-22,2.971579,-0.541053,7.477895,1.304211,6.088421,2.026316,65.282105,0.146316,0.146316,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
3,2018-01-23,-6.051579,-9.615789,0.953684,0.064211,7.265263,4.091579,48.290526,0.046316,0.170526,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
4,2018-01-24,-9.991579,-13.110526,-6.069474,0.206316,6.877895,3.460000,41.976842,0.303158,0.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,17072.0,6755.0,23.0,44644943.0,43999829.0,27237628.0,26798.0,87.02,85.76,53.09
1472,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,18333.0,6772.0,17.0,44645536.0,44000785.0,27256726.0,20646.0,87.02,85.76,53.13
1473,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,20268.0,6787.0,15.0,44645621.0,44000950.0,27259463.0,2987.0,87.02,85.76,53.13
1474,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,22904.0,6812.0,25.0,44646116.0,44001773.0,27271785.0,13640.0,87.02,85.76,53.16


In [85]:
final = final[final['일시'] != 0]
final

,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),일 최심신적설(cm),일 최심적설(cm),...,new_cases,total_deaths,new_deaths,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred
0,2018-01-20,1.284211,-3.844211,7.565263,0.000000,4.089474,1.457895,64.489474,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
1,2018-01-21,1.311579,-3.871579,7.481053,0.000000,3.431579,1.191579,60.866316,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
2,2018-01-22,2.971579,-0.541053,7.477895,1.304211,6.088421,2.026316,65.282105,0.146316,0.146316,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
3,2018-01-23,-6.051579,-9.615789,0.953684,0.064211,7.265263,4.091579,48.290526,0.046316,0.170526,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
4,2018-01-24,-9.991579,-13.110526,-6.069474,0.206316,6.877895,3.460000,41.976842,0.303158,0.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,2022-01-16,0.997895,-3.826316,5.397895,0.078947,6.332632,2.932632,54.527368,0.000000,0.236842,...,3858.0,6333.0,23.0,44509913.0,43532329.0,23364731.0,21121.0,86.76,84.85,45.54
1458,2022-01-17,-1.933684,-6.302105,2.733684,0.132632,6.289474,2.768421,54.464211,0.069474,0.238947,...,4070.0,6378.0,45.0,44525972.0,43603604.0,23750998.0,473538.0,86.79,84.99,46.29
1459,2022-01-18,-3.614737,-8.514737,2.106316,0.027368,4.697895,2.003158,46.534737,0.049474,0.236842,...,5800.0,6452.0,74.0,44538140.0,43650179.0,24013683.0,321370.0,86.81,85.08,46.81
1460,2022-01-19,-2.310526,-8.470526,3.992632,0.541053,5.502105,1.915789,59.974737,0.261053,0.462105,...,6601.0,6480.0,28.0,44548292.0,43695392.0,24229538.0,271154.0,86.83,85.17,47.23


In [88]:
#final.to_csv('final.csv', encoding = 'euc-kr')